In [65]:
#IMPORTs
import pandas as pd
import numpy as np

import os
os.chdir('/Users/danasouter/desktop/qbio490/qbio_490_dana/analysis_data')

import cptac

cptac.list_datasets()

cptac.download(dataset="ccrcc")

ccrcc = cptac.Ccrcc()

protein_data = ccrcc.get_proteomics()
rna_data = ccrcc.get_transcriptomics()
clinical_data = ccrcc.get_clinical()

In [66]:
#print(clinical_data.columns.tolist())


['Sample_Tumor_Normal', 'tumor/normal', 'gender', 'age', 'height_in_cm', 'height_in_inch', 'weight_in_kg', 'weight_in_lb', 'BMI', 'race', 'ethnicity', 'ethnicity_self_identified', 'tumor_site', 'tumor_site_other', 'tumor_size_in_cm', 'tumor_focality', 'histologic_type', 'histologic_type_other', 'histologic_grade', 'grading_system', 'tumor_stage_pathological', 'AJCC_or_TNM_cancer_staging_edition', 'sarcomatoid_features', 'sarcomatiod_percent', 'pathologic_staging_primary_tumor_pT', 'pathologic_staging_regional_lymph_nodes_pN', 'pathologic_staging_distant_metastasis_pM', 'clinical_staging_distant_metastasis_cM', 'serum_calcium', 'hemoglobin', 'platelets', 'white_cell_count', 'history_of_cancer', 'history_of_cancer_treatment', 'vital_status_at_12months_follow_up', 'vital_status_at_24months_follow_up', 'participant_in_US', 'participant_country_of_origin', 'tumor_laterality', 'tumor_necrosis', 'margin_status', 'number_of_lymph_nodes_examined', 'lymph_nodes_positive_for_tumor_by_IHC_staining

In [67]:
#1) Select what features to include in the model by finding the top 5 most differentially
#expressed proteins between Stage I and Stage III patients in CPTAC protein data. Repeat
#this process to find the top 5 most differential expression RNA between Stage I and Stage
#III patients in the CPTAC RNA data.

#a) Use tumor_stage_pathological in the CPTAC clinical data.

#//use if statements
if isinstance(protein_data.columns, pd.MultiIndex):
    protein_data.columns = protein_data.columns.get_level_values(0)
if isinstance(rna_data.columns, pd.MultiIndex):
    rna_data.columns = rna_data.columns.get_level_values(0)
    
protein_data_merged = protein_data.merge(clinical_data[['tumor_stage_pathological']], left_index=True, right_index=True)
rna_data_merged = rna_data.merge(clinical_data[['tumor_stage_pathological']], left_index=True, right_index=True)

stage_I_protein = protein_data_merged[protein_data_merged['tumor_stage_pathological'] == 'Stage I']
stage_III_protein = protein_data_merged[protein_data_merged['tumor_stage_pathological'] == 'Stage III']

stage_I_rna = rna_data_merged[rna_data_merged['tumor_stage_pathological'] == 'Stage I']
stage_III_rna = rna_data_merged[rna_data_merged['tumor_stage_pathological'] == 'Stage III']

# For Proteins
diff_proteins = abs(stage_I_protein.drop(['tumor_stage_pathological'], axis=1).mean() - stage_III_protein.drop(['tumor_stage_pathological'], axis=1).mean())
top_5_proteins = diff_proteins.nlargest(5).index.tolist()

# For RNA 
stage_I_rna_log = np.log2(stage_I_rna.drop(['tumor_stage_pathological'], axis=1) + 1)
stage_III_rna_log = np.log2(stage_III_rna.drop(['tumor_stage_pathological'], axis=1) + 1)
diff_rna = abs(stage_I_rna_log.mean() - stage_III_rna_log.mean())
top_5_rna = diff_rna.nlargest(5).index.tolist()


In [68]:
#2) Create a new dataframe of your selected features, where the rows are the patients and the
#columns are the expression values of genes you selected in step 1 (X data).

selected_features = top_5_proteins + top_5_rna

# For Proteins
selected_protein_data = protein_data_merged[top_5_proteins]

# For RNA
selected_rna_data = rna_data_merged[top_5_rna]

X_data = pd.concat([selected_protein_data, selected_rna_data], axis=1)
X_data_filled = X_data.fillna(X_data.mean())

#print(X_data_filled)

In [69]:
#3) Create a separate list of the patients’ cancer stages, ie. tumor_stage_pathological (y data).

y_data = clinical_data['tumor_stage_pathological']
y_data_aligned = y_data.loc[X_data.index]
y_data_list = y_data_aligned.tolist()


In [70]:
#4) Scale and encode your features and target.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_data)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_data_aligned)


In [71]:
#5) Create a train test split of your X and y data with train_size=0.7.

from sklearn.model_selection import train_test_split

# Split the data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, train_size=0.7, random_state=42)

#print(f"X_train shape: {X_train.shape}")
#print(f"X_test shape: {X_test.shape}")
#print(f"y_train shape: {y_train.shape}")
#print(f"y_test shape: {y_test.shape}")


In [72]:
#6) Write code to test the accuracy of all 4 classification models we covered in this class (ie.
#KNeighborsClassifier, DecisionTreeClassifier, and MLPClassifier, GaussianNB). Since
#the accuracy of the models will change depending on the train-test split, you will need to
#run each model 10 times and find the average accuracy between all runs.

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_data)

# Define 
models = {
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "MLPClassifier": MLPClassifier(max_iter=1000),  
    "GaussianNB": GaussianNB()
}

# # of runs
n_runs = 10

# accuracy 
accuracies = {model_name: [] for model_name in models}

# Run each 10 times
for run in range(n_runs):
    X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_encoded, train_size=0.7, random_state=run)

    for model_name, model in models.items():
        # Train model
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies[model_name].append(accuracy)

average_accuracies = {model_name: np.mean(acc) for model_name, acc in accuracies.items()}

for model_name, avg_acc in average_accuracies.items():
    print(f"{model_name}: Average Accuracy = {avg_acc:.4f}")



KNeighborsClassifier: Average Accuracy = 0.6983
DecisionTreeClassifier: Average Accuracy = 0.6373
MLPClassifier: Average Accuracy = 0.6475
GaussianNB: Average Accuracy = 0.5763


In [73]:
#7) Compare the 4 mean accuracies and identify which model is best.

best_model_name = max(average_accuracies, key=average_accuracies.get)
best_model_accuracy = average_accuracies[best_model_name]

print(f"\nThe best model is {best_model_name} with an Average Accuracy of {best_model_accuracy:.4f}.")


The best model is KNeighborsClassifier with an Average Accuracy of 0.6983.
